# Automating Crypto Website API Pull

Developing this project based on this [video.](https://www.youtube.com/watch?v=KB2CtEDrglY)

I am going to work with Coinmarketcap API:

https://coinmarketcap.com/api/

In [8]:
#Import libraries
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
import os
from time import time
from time import sleep

## API call

In [9]:
#Testing API is working -TAKEN FROM API Documentation

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'15', #Modify as you like 
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '68a74ed6-1adb-4562-8af1-a74d736dddf8', ##changed to my personal key
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

{'status': {'timestamp': '2025-01-30T09:31:33.007Z', 'error_code': 0, 'error_message': None, 'elapsed': 28, 'credit_count': 1, 'notice': None, 'total_count': 10775}, 'data': [{'id': 1, 'name': 'Bitcoin', 'symbol': 'BTC', 'slug': 'bitcoin', 'num_market_pairs': 11884, 'date_added': '2010-07-13T00:00:00.000Z', 'tags': ['mineable', 'pow', 'sha-256', 'store-of-value', 'state-channel', 'coinbase-ventures-portfolio', 'three-arrows-capital-portfolio', 'polychain-capital-portfolio', 'binance-labs-portfolio', 'blockchain-capital-portfolio', 'boostvc-portfolio', 'cms-holdings-portfolio', 'dcg-portfolio', 'dragonfly-capital-portfolio', 'electric-capital-portfolio', 'fabric-ventures-portfolio', 'framework-ventures-portfolio', 'galaxy-digital-portfolio', 'huobi-capital-portfolio', 'alameda-research-portfolio', 'a16z-portfolio', '1confirmation-portfolio', 'winklevoss-capital-portfolio', 'usv-portfolio', 'placeholder-ventures-portfolio', 'pantera-capital-portfolio', 'multicoin-capital-portfolio', 'par

Work around in case of having the error **IOPub data rate exceeded.**

Just go to Anaconda prompt and type the following:

`jupyter notebook --NotebookApp.iopub_data_rate_limit=1e10`

In [10]:
data #check structure of the data -we can see is a JSON file

{'status': {'timestamp': '2025-01-30T09:31:33.007Z',
  'error_code': 0,
  'error_message': None,
  'elapsed': 28,
  'credit_count': 1,
  'notice': None,
  'total_count': 10775},
 'data': [{'id': 1,
   'name': 'Bitcoin',
   'symbol': 'BTC',
   'slug': 'bitcoin',
   'num_market_pairs': 11884,
   'date_added': '2010-07-13T00:00:00.000Z',
   'tags': ['mineable',
    'pow',
    'sha-256',
    'store-of-value',
    'state-channel',
    'coinbase-ventures-portfolio',
    'three-arrows-capital-portfolio',
    'polychain-capital-portfolio',
    'binance-labs-portfolio',
    'blockchain-capital-portfolio',
    'boostvc-portfolio',
    'cms-holdings-portfolio',
    'dcg-portfolio',
    'dragonfly-capital-portfolio',
    'electric-capital-portfolio',
    'fabric-ventures-portfolio',
    'framework-ventures-portfolio',
    'galaxy-digital-portfolio',
    'huobi-capital-portfolio',
    'alameda-research-portfolio',
    'a16z-portfolio',
    '1confirmation-portfolio',
    'winklevoss-capital-portfoli

In [11]:
#Let's get the data in a more proper format
pd.set_option('display.max.columns', None) #display all the columns
df = pd.json_normalize(data['data'])
df

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,infinite_supply,platform,cmc_rank,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated,quote.USD.price,quote.USD.volume_24h,quote.USD.volume_change_24h,quote.USD.percent_change_1h,quote.USD.percent_change_24h,quote.USD.percent_change_7d,quote.USD.percent_change_30d,quote.USD.percent_change_60d,quote.USD.percent_change_90d,quote.USD.market_cap,quote.USD.market_cap_dominance,quote.USD.fully_diluted_market_cap,quote.USD.tvl,quote.USD.last_updated,platform.id,platform.name,platform.symbol,platform.slug,platform.token_address
0,1,Bitcoin,BTC,bitcoin,11884,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",2.100000e+07,1.981710e+07,1.981710e+07,False,NaN,1,NaN,NaN,None,2025-01-30T09:30:00.000Z,105262.892269,5.259283e+10,23.2991,0.075691,2.850648,2.967989,12.202980,8.584213,51.760695,2.086006e+12,58.4991,2.210521e+12,None,2025-01-30T09:30:00.000Z,NaN,NaN,NaN,NaN,NaN
1,1027,Ethereum,ETH,ethereum,9902,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.205163e+08,1.205163e+08,True,NaN,2,NaN,NaN,None,2025-01-30T09:30:00.000Z,3209.521193,2.432661e+10,23.2928,0.540727,2.698637,-0.152287,-5.068084,-13.141087,28.311389,3.867996e+11,10.8456,3.867996e+11,None,2025-01-30T09:30:00.000Z,NaN,NaN,NaN,NaN,NaN
2,52,XRP,XRP,xrp,1519,2013-08-04T00:00:00.000Z,"[medium-of-exchange, enterprise-solutions, xrp...",1.000000e+11,5.764121e+10,9.998659e+10,False,NaN,3,NaN,NaN,None,2025-01-30T09:31:00.000Z,3.108231,6.879058e+09,-15.9572,0.028488,0.932848,-0.044748,50.248954,61.333065,497.326750,1.791622e+11,5.0236,3.108231e+11,None,2025-01-30T09:31:00.000Z,NaN,NaN,NaN,NaN,NaN
3,825,Tether USDt,USDT,tether,115087,2015-02-25T00:00:00.000Z,"[stablecoin, asset-backed-stablecoin, ethereum...",NaN,1.394058e+11,1.425729e+11,True,NaN,4,NaN,NaN,None,2025-01-30T09:30:00.000Z,1.000017,1.007061e+11,17.0495,-0.011976,0.040146,0.050970,0.145470,-0.056320,0.097422,1.394082e+11,3.9089,1.425754e+11,None,2025-01-30T09:30:00.000Z,1027.0,Ethereum,ETH,ethereum,0xdac17f958d2ee523a2206206994597c13d831ec7
4,5426,Solana,SOL,solana,834,2020-04-10T00:00:00.000Z,"[pos, platform, solana-ecosystem, cms-holdings...",NaN,4.867429e+08,5.930049e+08,True,NaN,5,NaN,NaN,None,2025-01-30T09:30:00.000Z,240.958748,6.340879e+09,42.0629,0.657348,3.871601,-2.821725,25.044964,1.565289,45.098141,1.172850e+11,3.2884,1.428897e+11,None,2025-01-30T09:30:00.000Z,NaN,NaN,NaN,NaN,NaN
5,1839,BNB,BNB,bnb,2345,2017-07-25T00:00:00.000Z,"[marketplace, centralized-exchange, payments, ...",NaN,1.424808e+08,1.424808e+08,False,NaN,6,NaN,NaN,None,2025-01-30T09:31:00.000Z,678.306162,1.636342e+09,-7.4212,0.159999,1.273487,-1.243256,-3.594262,3.459834,17.534866,9.664558e+10,2.7098,9.664558e+10,None,2025-01-30T09:31:00.000Z,NaN,NaN,NaN,NaN,NaN
6,3408,USDC,USDC,usd-coin,25279,2018-10-08T00:00:00.000Z,"[medium-of-exchange, stablecoin, asset-backed-...",NaN,5.266517e+10,5.266517e+10,False,NaN,7,NaN,NaN,None,2025-01-30T09:30:00.000Z,1.000065,9.582460e+09,22.8212,-0.007542,0.008803,0.007112,-0.007858,-0.000195,0.008076,5.266861e+10,1.4768,5.266861e+10,None,2025-01-30T09:30:00.000Z,1027.0,Ethereum,ETH,ethereum,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
7,74,Dogecoin,DOGE,dogecoin,1158,2013-12-15T00:00:00.000Z,"[mineable, pow, scrypt, medium-of-exchange, me...",NaN,1.478356e+11,1.478356e+11,True,NaN,8,NaN,NaN,None,2025-01-30T09:30:00.000Z,0.332124,1.923832e+09,0.9749,0.495476,1.502970,-5.600197,4.672917,-22.203036,109.680474,4.909983e+10,1.3767,4.909983e+10,None,2025-01-30T09:30:00.000Z,NaN,NaN,NaN,NaN,NaN
8,2010,Cardano,ADA,cardano,1453,2017-10-01T00:00:00.000Z,"[dpos, pos, platform, research, smart-contract...",4.500000e+10,3.518584e+10,4.499520e+10,False,NaN,9,NaN,NaN,None,2025-01-30T09:31:00.000Z,0.963352,8.061510e+08,8.9072,0.391662,3.868121,-0.013987,12.183342,-10.000320,178.455875,3.389635e+10,0.9506,4.335083e+10,None,2025-01-30T09:31:00.000Z,NaN,N

In [12]:
df.shape #Matriz size

(15, 36)

In [13]:
#IMPORTANT: First thing to be done is gonna be add a TIMESTAMP, which allows to identify when the data is retrieved
df['timestamp'] = pd.Timestamp('now')
df

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,infinite_supply,platform,cmc_rank,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated,quote.USD.price,quote.USD.volume_24h,quote.USD.volume_change_24h,quote.USD.percent_change_1h,quote.USD.percent_change_24h,quote.USD.percent_change_7d,quote.USD.percent_change_30d,quote.USD.percent_change_60d,quote.USD.percent_change_90d,quote.USD.market_cap,quote.USD.market_cap_dominance,quote.USD.fully_diluted_market_cap,quote.USD.tvl,quote.USD.last_updated,platform.id,platform.name,platform.symbol,platform.slug,platform.token_address,timestamp
0,1,Bitcoin,BTC,bitcoin,11884,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",2.100000e+07,1.981710e+07,1.981710e+07,False,NaN,1,NaN,NaN,None,2025-01-30T09:30:00.000Z,105262.892269,5.259283e+10,23.2991,0.075691,2.850648,2.967989,12.202980,8.584213,51.760695,2.086006e+12,58.4991,2.210521e+12,None,2025-01-30T09:30:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-01-30 10:31:36.437661
1,1027,Ethereum,ETH,ethereum,9902,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.205163e+08,1.205163e+08,True,NaN,2,NaN,NaN,None,2025-01-30T09:30:00.000Z,3209.521193,2.432661e+10,23.2928,0.540727,2.698637,-0.152287,-5.068084,-13.141087,28.311389,3.867996e+11,10.8456,3.867996e+11,None,2025-01-30T09:30:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-01-30 10:31:36.437661
2,52,XRP,XRP,xrp,1519,2013-08-04T00:00:00.000Z,"[medium-of-exchange, enterprise-solutions, xrp...",1.000000e+11,5.764121e+10,9.998659e+10,False,NaN,3,NaN,NaN,None,2025-01-30T09:31:00.000Z,3.108231,6.879058e+09,-15.9572,0.028488,0.932848,-0.044748,50.248954,61.333065,497.326750,1.791622e+11,5.0236,3.108231e+11,None,2025-01-30T09:31:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-01-30 10:31:36.437661
3,825,Tether USDt,USDT,tether,115087,2015-02-25T00:00:00.000Z,"[stablecoin, asset-backed-stablecoin, ethereum...",NaN,1.394058e+11,1.425729e+11,True,NaN,4,NaN,NaN,None,2025-01-30T09:30:00.000Z,1.000017,1.007061e+11,17.0495,-0.011976,0.040146,0.050970,0.145470,-0.056320,0.097422,1.394082e+11,3.9089,1.425754e+11,None,2025-01-30T09:30:00.000Z,1027.0,Ethereum,ETH,ethereum,0xdac17f958d2ee523a2206206994597c13d831ec7,2025-01-30 10:31:36.437661
4,5426,Solana,SOL,solana,834,2020-04-10T00:00:00.000Z,"[pos, platform, solana-ecosystem, cms-holdings...",NaN,4.867429e+08,5.930049e+08,True,NaN,5,NaN,NaN,None,2025-01-30T09:30:00.000Z,240.958748,6.340879e+09,42.0629,0.657348,3.871601,-2.821725,25.044964,1.565289,45.098141,1.172850e+11,3.2884,1.428897e+11,None,2025-01-30T09:30:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-01-30 10:31:36.437661
5,1839,BNB,BNB,bnb,2345,2017-07-25T00:00:00.000Z,"[marketplace, centralized-exchange, payments, ...",NaN,1.424808e+08,1.424808e+08,False,NaN,6,NaN,NaN,None,2025-01-30T09:31:00.000Z,678.306162,1.636342e+09,-7.4212,0.159999,1.273487,-1.243256,-3.594262,3.459834,17.534866,9.664558e+10,2.7098,9.664558e+10,None,2025-01-30T09:31:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-01-30 10:31:36.437661
6,3408,USDC,USDC,usd-coin,25279,2018-10-08T00:00:00.000Z,"[medium-of-exchange, stablecoin, asset-backed-...",NaN,5.266517e+10,5.266517e+10,False,NaN,7,NaN,NaN,None,2025-01-30T09:30:00.000Z,1.000065,9.582460e+09,22.8212,-0.007542,0.008803,0.007112,-0.007858,-0.000195,0.008076,5.266861e+10,1.4768,5.266861e+10,None,2025-01-30T09:30:00.000Z,1027.0,Ethereum,ETH,ethereum,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,2025-01-30 10:31:36.437661
7,74,Dogecoin,DOGE,dogecoin,1158,2013-12-15T00:00:00.000Z,"[mineable, pow, scrypt, medium-of-exchange, me...",NaN,1.478356e+11,1.478356e+11,True,NaN,8,NaN,NaN,None,2025-01-30T09:30:00.000Z,0.332124,1.923832e+09,0.9749,0.495476,1.502970,-5.600197,4.672917,-22.203036,109.680474,4.909983e+10,1.3767,4.909983e+10,None,2025-01-30T09:30:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-01-30 10:31:36.437661
8,2010,Cardano,ADA,cardano,1453,2017-10-01T00:00:00.000Z,"[dpos, pos, platform, research, smart-contract...",4.500000e+10,3.51858

In [14]:
#Now: We are going to create an automated script to execute the API pull
#1. We define the function

def api_runner():
    global df #we set as a global variable
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'15', #Modify as you like 
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '68a74ed6-1adb-4562-8af1-a74d736dddf8', ##my personal key
    }

    session = Session()
    session.headers.update(headers)

    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      print('Success retrieval')
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)
    
    #We define df2 to append the new information retrieved into df defined before
    
    # Process data
    df = pd.json_normalize(data['data'])
    df['timestamp'] = pd.Timestamp.now()
    df
        
    #Save into csv file
    if not os.path.isfile(r'C:\Users\Leo\Desktop\Python Projects\ETL-ELT-examples\Automating-Crypto-Website-API-Pull\API.csv'): #check if not exist
        df.to_csv(r'C:\Users\Leo\Desktop\Python Projects\ETL-ELT-examples\Automating-Crypto-Website-API-Pull\API.csv', header ='column_names')
    else:
        df.to_csv(r'C:\Users\Leo\Desktop\Python Projects\ETL-ELT-examples\Automating-Crypto-Website-API-Pull\API.csv', mode ='a', header=False) #mode stands for append

In [6]:
#2. We automate the script to be executed on time ranges

for i in range(3): #I am setting 333 because that is the max calls Coinmarketcap allows me to make per day
    api_runner()
    print('API runner completed!')
    sleep(60) #sleep for 1 minuto
exit()

#This loop is executing itself 333 times. It runs api_runner, then print 'API runner completed!' and waits 60 seconds to execute the next

Success retrieval
API runner completed!
Success retrieval
API runner completed!
Success retrieval
API runner completed!


KeyboardInterrupt: 

In [7]:
#Just checking the function made what actually is expected to do
df_csv = pd.read_csv(r'C:\Users\Leo\Desktop\Python Projects\ETL-ELT-examples\Automating-Crypto-Website-API-Pull\API.csv')
df_csv

,Unnamed: 0,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,...,quote.USD.market_cap_dominance,quote.USD.fully_diluted_market_cap,quote.USD.tvl,quote.USD.last_updated,platform.id,platform.name,platform.symbol,platform.slug,platform.token_address,timestamp
0,0,1,Bitcoin,BTC,bitcoin,11884,2010-07-13T00:00:00.000Z,"['mineable', 'pow', 'sha-256', 'store-of-value...",2.100000e+07,1.981710e+07,...,58.4828,2.209921e+12,NaN,2025-01-30T09:24:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-01-30 10:26:47.335203
1,1,1027,Ethereum,ETH,ethereum,9902,2015-08-07T00:00:00.000Z,"['pos', 'smart-contracts', 'ethereum-ecosystem...",NaN,1.205163e+08,...,10.8524,3.869304e+11,NaN,2025-01-30T09:25:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-01-30 10:26:47.335203
2,2,52,XRP,XRP,xrp,1519,2013-08-04T00:00:00.000Z,"['medium-of-exchange', 'enterprise-solutions',...",1.000000e+11,5.764121e+10,...,5.0229,3.106935e+11,NaN,2025-01-30T09:25:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-01-30 10:26:47.335203
3,3,825,Tether USDt,USDT,tether,115087,2015-02-25T00:00:00.000Z,"['stablecoin', 'asset-backed-stablecoin', 'eth...",NaN,1.394058e+11,...,3.9099,1.425698e+11,NaN,2025-01-30T09:25:00.000Z,1027.0,Ethereum,ETH,ethereum,0xdac17f958d2ee523a2206206994597c13d831ec7,2025-01-30 10:26:47.335203
4,4,5426,Solana,SOL,solana,834,2020-04-10T00:00:00.000Z,"['pos', 'platform', 'solana-ecosystem', 'cms-h...",NaN,4.867429e+08,...,3.2890,1.428604e+11,NaN,2025-01-30T09:25:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-01-30 10:26:47.335203
5,5,1839,BNB,BNB,bnb,2345,2017-07-25T00:00:00.000Z,"['marketplace', 'centralized-exchange', 'payme...",NaN,1.424808e+08,...,2.7115,9.667018e+10,NaN,2025-01-30T09:25:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-01-30 10:26:47.335203
6,6,3408,USDC,USDC,usd-coin,25277,2018-10-08T00:00:00.000Z,"['medium-of-exchange', 'stablecoin', 'asset-ba...",NaN,5.266517e+10,...,1.4771,5.266338e+10,NaN,2025-01-30T09:25:00.000Z,1027.0,Ethereum,ETH,ethereum,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,2025-01-30 10:26:47.335203
7,7,74,Dogecoin,DOGE,dogecoin,1158,2013-12-15T00:00:00.000Z,"['mineable', 'pow', 'scrypt', 'medium-of-excha...",NaN,1.478356e+11,...,1.3767,4.908565e+10,NaN,2025-01-30T09:25:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-01-30 10:26:47.335203
8,8,2010,Cardano,ADA,cardano,1453,2017-10-01T00:00:00.000Z,"['dpos', 'pos', 'platform', 'research', 'smart...",4.500000e+10,3.518584e+10,...,0.9495,4.330430e+10,NaN,2025-01-30T09:24:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-01-30 10:26:47.335203
9,9,1958,TRON,TRX,tron,1113,2017-09-13T00:00:00.000Z,"['media', 'payments', 'ethereum-ecosystem', 't...",NaN,8.612894e+10,...,0.5978,2.131496e+10,NaN,2025-01-30T09:25:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-01-30 10:26:47.335203


## Data Cleaning

In [15]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [16]:
df_csv

,Unnamed: 0,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,infinite_supply,platform,cmc_rank,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated,quote.USD.price,quote.USD.volume_24h,quote.USD.volume_change_24h,quote.USD.percent_change_1h,quote.USD.percent_change_24h,quote.USD.percent_change_7d,quote.USD.percent_change_30d,quote.USD.percent_change_60d,quote.USD.percent_change_90d,quote.USD.market_cap,quote.USD.market_cap_dominance,quote.USD.fully_diluted_market_cap,quote.USD.tvl,quote.USD.last_updated,platform.id,platform.name,platform.symbol,platform.slug,platform.token_address,timestamp
0,0,1,Bitcoin,BTC,bitcoin,11884,2010-07-13T00:00:00.000Z,"['mineable', 'pow', 'sha-256', 'store-of-value...",21000000.00000,19817103.00000,19817103.00000,False,NaN,1,NaN,NaN,NaN,2025-01-30T09:24:00.000Z,105234.34302,52660656818.75706,23.44980,0.06959,2.71832,2.86488,12.04167,8.57065,51.72728,2085439814835.96631,58.48280,2209921203495.54980,NaN,2025-01-30T09:24:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-01-30 10:26:47.335203
1,1,1027,Ethereum,ETH,ethereum,9902,2015-08-07T00:00:00.000Z,"['pos', 'smart-contracts', 'ethereum-ecosystem...",NaN,120516309.23334,120516309.23334,True,NaN,2,NaN,NaN,NaN,2025-01-30T09:25:00.000Z,3210.60583,24685029145.66537,23.55840,0.53568,2.50905,-0.07973,-5.18701,-13.18119,28.36734,386930365163.34930,10.85240,386930365163.34998,NaN,2025-01-30T09:25:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-01-30 10:26:47.335203
2,2,52,XRP,XRP,xrp,1519,2013-08-04T00:00:00.000Z,"['medium-of-exchange', 'enterprise-solutions',...",100000000000.00000,57641211234.00000,99986586496.00000,False,NaN,3,NaN,NaN,NaN,2025-01-30T09:25:00.000Z,3.10693,6894874179.76889,-15.76000,-0.07010,0.71523,-0.14918,49.88084,60.98909,496.40900,179087488585.69815,5.02290,310693486052.32001,NaN,2025-01-30T09:25:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-01-30 10:26:47.335203
3,3,825,Tether USDt,USDT,tether,115087,2015-02-25T00:00:00.000Z,"['stablecoin', 'asset-backed-stablecoin', 'eth...",NaN,139405758247.31223,142572893912.02841,True,NaN,4,NaN,NaN,NaN,2025-01-30T09:25:00.000Z,0.99998,100794903640.32265,17.20600,-0.01066,0.03308,0.05337,0.16288,-0.05566,0.09058,139402750158.86768,3.90990,142569817483.34000,NaN,2025-01-30T09:25:00.000Z,1027.00000,Ethereum,ETH,ethereum,0xdac17f958d2ee523a2206206994597c13d831ec7,2025-01-30 10:26:47.335203
4,4,5426,Solana,SOL,solana,834,2020-04-10T00:00:00.000Z,"['pos', 'platform', 'solana-ecosystem', 'cms-h...",NaN,486742946.24498,593004890.68357,True,NaN,5,NaN,NaN,NaN,2025-01-30T09:25:00.000Z,240.90929,6346582935.64289,42.08370,0.61874,3.54119,-2.88998,24.99979,1.52114,45.12632,117260898985.56155,3.28900,142860388878.42999,NaN,2025-01-30T09:25:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-01-30 10:26:47.335203
5,5,1839,BNB,BNB,bnb,2345,2017-07-25T00:00:00.000Z,"['marketplace', 'centralized-exchange', 'payme...",NaN,142480756.83000,142480756.83000,False,NaN,6,NaN,NaN,NaN,2025-01-30T09:25:00.000Z,678.47885,1635651403.80962,-7.60190,0.18430,1.18167,-1.25738,-3.44618,3.39253,17.61445,96670179832.42340,2.71150,96670179832.42000,NaN,2025-01-30T09:25:00.000Z,NaN,NaN,NaN,NaN,NaN,2025-01-30 10:26:47.335203
6,6,3408,USDC,USDC,usd-coin,25277,2018-10-08T00:00:00.000Z,"['medium-of-exchange', 'stablecoin', 'asset-ba...",NaN,52665172618.91838,52665172618.91838,False,NaN,7,NaN,NaN,NaN,2025-01-30T09:25:00.000Z,0.99997,9937337384.57644,23.53730,-0.01044,0.00163,0.00053,-0.00004,-0.00583,-0.00354,52663377218.84874,1.47710,52663377218.85000,NaN,2025-01-30T09:25:00.000Z,1027.00000,Ethereum,ETH,ethereum,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,2025-01-30 10:26:47.335203
7,7,74,Dogecoin,DOGE,dogecoin,1158,2013-12-15T00:00:00.000Z,"['mineable', 'pow', 'scrypt', 'medium-of-excha...",NaN,147835636383.70520,147835636383.70520,True,NaN,8,NaN,NaN,NaN,2025-01-30T09:25:00.000Z,0.33203,1929859789.45934,1.55230,0.43203,1.28812,-5.67124,4.53063,-22.27742,109.47054,49085651983.07829,1.37670,49085651983.08000,NaN,2025-01-30T09:25:0